In [1]:
import sys
sys.path.append('../../../src')
from helpers import *

In [2]:
df = pd.read_csv('../../Raw analysis/PAR-2 NEBD quantification/NEBD_Embryos_dataframe.csv')
df_quantification = pd.read_csv('../../Raw analysis/PAR-2 NEBD quantification/QUANTIFICATION.csv')
df_quantification_grouped = df_quantification.groupby('EmbryoID')

df_summary = pd.DataFrame()
for i, r in df_quantification_grouped:
    cyt = np.mean(r['Cytoplasmic signal'])
    mem_tot = np.mean(r['Membrane signal'])
    mem_ant = bounded_mean_1d(r['Membrane signal'], [0.4, 0.6])
    mem_post = bounded_mean_1d(r['Membrane signal'], [0.9, 0.1]) 
    mem_frac = mem_tot / ((cyt / 0.174) + mem_tot)
    ratio_post = mem_post / cyt
    dosage = cyt + (mem_tot * 0.174)
    p = {'EmbryoID': i, 'Cyt': cyt, 'Mem_tot': mem_tot, 'Mem_ant': mem_ant, 'Mem_post': mem_post,
        'Ratio post': ratio_post, 'Mem frac': mem_frac, 'Dosage': dosage} 
    df_summary = df_summary.append(pd.DataFrame(p, index=[i]))
df = pd.merge(df, df_summary, on='EmbryoID')

# Filter
lines_order = ['lp637', 'nwg201', 'nwg240', 'nwg246', 'nwg338', 'nwg369']
df = df[df.Line.isin(lines_order)]
df = df[df.RNAi == 'wt']

# Add uniform vs polarised column
unipol = ['Pol' if x in ['lp637', 'nwg240', 'nwg338'] else 'Uni' for x in df.Line]
df['UniPol'] = unipol

# Add genotype column
p2 = ['WT' if x in ['lp637', 'nwg201'] else 'C56S' if x in ['nwg240', 'nwg246'] else 'L109R' for x in df.Line]
df['P2_allele'] = p2

# Add label column
label_dict = {'lp637': 'PAR-2 (WT)', 
          'nwg201' : 'PAR-2 (WT); par-3 (it71)', 
          'nwg240': 'PAR-2 (C56S)', 
          'nwg246': 'PAR-2 (C56S); par-3 (it71)', 
          'nwg338': 'PAR-2 (L109R)', 
          'nwg369': 'PAR-2 (L109R); par-3 (it71)'}
df['Genotype'] = [label_dict[x] for x in df.Line]

# Normalise
norm = df[df.Line == 'lp637'].Cyt.mean()
df.Cyt /= norm
df.Mem_tot /= norm
df.Mem_ant /= norm
df.Mem_post /= norm
df.Dosage /= norm

In [3]:
def create_table(df):
    means = df.groupby('Genotype').mean()[['Cyt', 'Mem_post', 'Ratio post', 'Mem frac', 'Dosage']]
    means = means.rename(columns={'Cyt': 'Cyt_mean', 'Mem_post': 'Mem_mean', 'Ratio post': 'Ratio_mean',
                                 'Mem frac': 'Memfrac_mean', 'Dosage': 'Dosage_mean'})
    stds = df.groupby('Genotype').std()[['Cyt', 'Mem_post', 'Ratio post', 'Mem frac', 'Dosage']]
    stds = stds.rename(columns={'Cyt': 'Cyt_std', 'Mem_post': 'Mem_std', 'Ratio post': 'Ratio_std',
                               'Mem frac': 'Memfrac_std', 'Dosage': 'Dosage_std'})
    a = pd.concat([means, stds], axis=1)
    
    c1 = a.apply(lambda x: f'{x["Cyt_mean"]:.2f} ± {x["Cyt_std"]:.2f}', axis=1)
    c2 = a.apply(lambda x: f'{x["Mem_mean"]:.2f} ± {x["Mem_std"]:.2f}', axis=1)
    c3 = a.apply(lambda x: f'{x["Ratio_mean"]:.2f} ± {x["Ratio_std"]:.2f}', axis=1)
    c4 = a.apply(lambda x: f'{x["Memfrac_mean"]:.2f} ± {x["Memfrac_std"]:.2f}', axis=1)
    c5 = a.apply(lambda x: f'{x["Dosage_mean"]:.2f} ± {x["Dosage_std"]:.2f}', axis=1)

    RESULTS = pd.DataFrame({'Normalised cytoplasmic concentration (μm⁻³)': c1, 
                            'Normalised posterior membrane concentration (μm⁻²)': c2, 
                            'M:C ratio (μm)': c3,
                            'Fraction at membrane': c4,
                            'Total expression (μm⁻³)': c5})
    RESULTS = RESULTS.reindex([label_dict[i] for i in lines_order])
    return RESULTS

In [4]:
res = create_table(df)

In [5]:
res.to_csv('results_table.csv')